In [28]:
from ipycanvas import Canvas

In [31]:
import dinkum
from dinkum.vfg import Gene
from dinkum.vfn import Tissue
from dinkum import Timecourse
from dinkum import observations

In [53]:
def is_active(time_point, gene):
    return True

def draw_tissue(tissue_num, times, gene_names, is_active_fn):
    active_color = "DeepSkyBlue"
    inactive_color = "DarkGrey"
    
    assert tissue_num == 0

    box_x_start = 100
    box_y_start = 100

    box_size = 25
    box_spacing = 5

    ##

    box_total_size = box_size + box_spacing

    for row in range(0, len(times)):
        for col in range(0, len(gene_names)):
            if is_active_fn(times[row], gene_names[col]):
                canvas.fill_style = active_color
            else:
                canvas.fill_style = inactive_color

            xpos = box_x_start + box_total_size*col
            ypos = box_y_start + box_total_size*row
            canvas.fill_rect(xpos, ypos, box_size, box_size)


    canvas.font = "18px Arial"
    canvas.text_baseline = "top"
    canvas.fill_style = "black"

    # row names / time points
    canvas.text_align = "right"
    for row in range(0, len(times)):
        xpos = box_x_start - box_total_size / 2
        ypos = box_y_start + box_total_size*row
        canvas.fill_text(times[row], xpos, ypos)

    # col names / genes
    canvas.text_align = "center"
    for col in range(0, len(gene_names)):
        ypos = box_y_start - box_total_size
        xpos = box_x_start + box_total_size*col + box_total_size / 2

        canvas.fill_text(gene_names[col], xpos, ypos, max_width = box_total_size)
        
canvas = Canvas(width=500, height=500)

draw_tissue(0, [ f"t{x}" for x in range(10) ], [ 'a', 'b', 'c', 'd', 'e' ], is_active)

canvas

Canvas(width=500)

In [73]:
def tc_record_activity(*, start=0, stop=10, gene_names=None, tissue_name=None, verbose=False):
    assert tissue_name is not None, "must specify tissue"
    
    tc = dinkum.Timecourse()

    activity_record = []

    times = []
    time_points = {}
    for n, state in enumerate(tc.iterate(start=start, stop=stop)):
        tp = f"t={state.time}"
        if verbose:
            print(tp)
        times.append(tp)
        time_points[tp] = n

        for ti in state.tissues:
            present = state[ti]
            if verbose:
                print(f"\ttissue={ti.name}, {present.report_activity()}")
            if ti.name == tissue_name:
                activity_record.append(present)

    def is_active(time_point, gene):
        time_idx = time_points[time_point]
        ga = activity_record[time_idx]
        active = ga.is_active(gene)
        return bool(active)
    
    if gene_names is None:
        gene_names = list(sorted(present.genes_by_name))
    
    return times, gene_names, is_active

In [74]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.activated_by_and(sources=[x, y])

m = Tissue(name='M')
x.is_present(where=m, start=1)

times, genes, is_active_fn = tc_record_activity(tissue_name="M", stop=5)

canvas = Canvas(width=500, height=500)

draw_tissue(0, times, genes, is_active_fn)

canvas

got 1 tissues.
	tissue M


Canvas(width=500)

In [78]:
dinkum.reset()

x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.and_not(activator=x, repressor=y)

m = Tissue(name='M')
x.is_present(where=m, start=1)

# run!                                                                      
times, genes, is_active_fn = tc_record_activity(tissue_name="M", stop=5)

canvas = Canvas(width=500, height=500)
draw_tissue(0, times, genes, is_active_fn)
canvas

got 1 tissues.
	tissue M


Canvas(width=500)

In [79]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')
s = Gene(name='S')          # switches spec states b/t tissues M and N      

y.activated_by_and(sources=[x, s])
z.and_not(activator=x, repressor=y)

m = Tissue(name='M')
x.is_present(where=m, start=1)

m = Tissue(name='N')
x.is_present(where=m, start=1)
s.is_present(where=m, start=1)

# run!                                                                      
times, genes, is_active_fn = tc_record_activity(tissue_name="N", stop=5)

canvas = Canvas(width=500, height=500)
draw_tissue(0, times, genes, is_active_fn)
canvas

got 2 tissues.
	tissue M
	tissue N


Canvas(width=500)